# Create chunked dataset

- LJSpeech 

- id 
- transcription
- normalized transcription

In [4]:
import pandas as pd 
import pathlib 
from pathlib import Path
from typing import List, Dict, Union
DATA_PATH: str = './data/'

In [5]:
MP3_FILES: List =  list(pathlib.Path(DATA_PATH).rglob('*.mp3'))
VTT_FILES: List = list(pathlib.Path(DATA_PATH).rglob('*.vtt'))

In [6]:
with open(VTT_FILES[0], 'r') as outfile:
    data = outfile.readlines()

In [7]:
times = data[2::3]
speeches = data[3::3]

In [8]:
ids = list(range(len(speeches)))

In [31]:
speeches_frame = pd.DataFrame([ids, speeches]).T
speeches_frame.columns = ['ID', 'transcription']

In [33]:
times_frame  = pd.DataFrame([ids, times]).T
times_frame.columns = ['ID', 'times']

In [35]:
import re
_from = [el[0] for el in  times_frame.times.apply(lambda x: re.sub('\n', '', x).split('-->'))]
_to = [el[1] for el in  times_frame.times.apply(lambda x: re.sub('\n', '', x).split('-->'))]
times_frame['from'] = _from 
times_frame['to'] = _to

In [36]:
_from =   ['00:' + el.strip() if len(el) == 10 else el for el in _from]
_to  = ['00:' + el.strip() if len(el) == 10 else el for el in _to]

In [37]:
from datetime import datetime
_to_duration = [(datetime.strptime(el.strip(),   '%H:%M:%S.%f')  - datetime.strptime(_from[0].strip(), '%H:%M:%S.%f')).total_seconds() for el in _to]
_from_duration = [(datetime.strptime(el.strip(),   '%H:%M:%S.%f')  - datetime.strptime(_from[0].strip(), '%H:%M:%S.%f')).total_seconds() for el in _from ]

In [38]:
speeches_frame.columns

Index(['ID', 'transcription'], dtype='object')

In [40]:
speeches_frame.ID = speeches_frame.ID.apply(lambda x: f'./data/wavs/become_backprop_ninja_{x}.wav')

In [42]:
speeches_frame.to_csv('./data/become_backprop.csv', index=False)

In [48]:
#data = speeches_frame.ID + '|' + speeches_frame.transcription

In [47]:
data.to_csv(r'./data/become_backprop.txt', header=None, index=None, sep='\t', mode='a')

# Read that fucking mp3 file 

In [92]:
import librosa
wav, sr = librosa.load('data/becoming_a_backprop_ninja.wav')

In [93]:
import soundfile as sf

In [95]:
sr = 22050
for el in range(len(_to_duration)):
    tmp_ = wav[int(_from_duration[el] * sr): int(_to_duration[el] * sr)]
    sf.write(f'./data/become_backprop_ninja_{el}.wav', tmp_, sr)

array([0.       , 0.       , 0.       , ..., 0.0859375, 0.0703125,
       0.0546875], dtype=float32)

## Set up naturalspeech

In [ ]:
%cd /content/
!git clone https://github.com/heatz123/naturalspeech
%cd /content/naturalspeech
!pip install -r requirements.txt
%cd /content/
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 

## Create symbolic link to dataset

In [ ]:
%cd /content/
!tar -xf LJSpeech-1.1.tar.bz2
%cd /content/naturalspeech
!ln -s /content/LJSpeech-1.1/wavs/ DUMMY1

## Uncompress durations labels

In [ ]:
!mv /content/naturalspeech/durations/durations.tar.bz2 /content/naturalspeech/
!rm -r /content/naturalspeech/durations
%cd  /content/naturalspeech/
!tar -xf durations.tar.bz2

## Warmup Training

In [ ]:
%cd /content/naturalspeech
!python3 train.py -c configs/ljs.json -m exp1 --warmup

## Training

In [ ]:
!python3 attach_memory_bank.py -c configs/ljs.json --weights_path logs/ext/G_500.pth

In [ ]:
!python3 train.py -c configs/ljs.json -m exp1